## Google drive와 Google Colab의 연동
-  colab에서 작업 시 google drive에 저장되어 있는 데이터에 접근하기 위해서는 colab을 연동시키는 작업이 필요함.

1. 아래의 코드를 실행.

In [1]:
!apt-get install -y -qq software-properties-common python-software-properties module-init-tools
!add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null
!apt-get update -qq 2>&1 > /dev/null
!apt-get -y install -qq google-drive-ocamlfuse fuse

from google.colab import auth
auth.authenticate_user()

from oauth2client.client import GoogleCredentials
creds = GoogleCredentials.get_application_default()

import getpass
!google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret} < /dev/null 2>&1 | grep URL
vcode = getpass.getpass()
!echo {vcode} | google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret}

E: Package 'python-software-properties' has no installation candidate
Selecting previously unselected package google-drive-ocamlfuse.
(Reading database ... 125048 files and directories currently installed.)
Preparing to unpack .../google-drive-ocamlfuse_0.7.14-0ubuntu1~ubuntu18.04.1_amd64.deb ...
Unpacking google-drive-ocamlfuse (0.7.14-0ubuntu1~ubuntu18.04.1) ...
Setting up google-drive-ocamlfuse (0.7.14-0ubuntu1~ubuntu18.04.1) ...
Processing triggers for man-db (2.8.3-2ubuntu0.1) ...
Please, open the following URL in a web browser: https://accounts.google.com/o/oauth2/auth?client_id=32555940559.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive&response_type=code&access_type=offline&approval_prompt=force
··········
Please, open the following URL in a web browser: https://accounts.google.com/o/oauth2/auth?client_id=32555940559.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope

2. 코드 실행 중 secret key를 입력하라는 빈 칸이 나오면 링크를 클릭하여 구글 아이디로 로그인 하고 key를 발급받아  복사/붙이기로 칸에 입력하고 엔터를 누름(두번 반복).  "Access token retrieved correctly."라는 결과가 나오는 것을 확인.

3. 아래의 코드를 실행하여 "drive" 이름의 폴더를 생성하고 이 폴더를 구글 드라이브와 연동.

In [0]:
!mkdir -p drive
!google-drive-ocamlfuse drive

4. drive 폴더에서 google drive에 있는 파일이 접근되는지 확인.


In [3]:
!cd drive; ls

 시각인공지능	        data				       Untitled1.ipynb
 aesop.csv	        Deep_Learning_한국정보문화산업진흥원   Untitled2.ipynb
 case03_고객분석        fine_tuning.ipynb		       Untitled3.ipynb
'Colab Notebooks'      'Getting started'		       YOLO
 CRAFT-pytorch-master   Untitled0.ipynb


- 일정 시간이 지나면 연동이 해제되므로 다시 실행해 주어야 함.

## Tensorboard 사용하기
- 연동된 google drive 상의 폴더에 tensorboard 실행 시의 log 파일을 저장하고 이를 확인하는 예제 

#### Import data

In [4]:
from keras.datasets import mnist   
from keras.utils import to_categorical

(train_images, train_labels), (test_images, test_labels) = mnist.load_data()

train_images = train_images.reshape((60000, 28, 28, 1))
train_images = train_images.astype('float32') / 255

test_images = test_images.reshape((10000, 28, 28, 1))
test_images = test_images.astype('float32') / 255

train_labels = to_categorical(train_labels)
test_labels = to_categorical(test_labels)

Using TensorFlow backend.


11493376/11490434 [==============================] - 1s 0us/step


In [0]:
train_images.shape

#### `LOG_DIR`에 log를 기록할 위치를 지정하고 tensorboard의 실행 port에 접근할 수 있는 링크 생성
- Google drive의 `data/tb_logs` 폴더에 저장하도록 지정함.
- Tensorboard를 실행하고 나면 Google drive에서 해당 폴더가 생성되어 있고 그 안에 log 파일이 있는 것이 확인됨.

In [8]:

LOG_DIR = '/content/drive/data/tb_logs'  

!wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
!unzip ngrok-stable-linux-amd64.zip

import os
if not os.path.exists(LOG_DIR):
  os.makedirs(LOG_DIR)

get_ipython().system_raw(
    'tensorboard --logdir {} --host 0.0.0.0 --port 6006 &'
    .format(LOG_DIR))

get_ipython().system_raw('./ngrok http 6006 &')

!curl -s http://localhost:4040/api/tunnels | python3 -c \
    "import sys, json; print(json.load(sys.stdin)['tunnels'][0]['public_url'])"

--2019-12-09 14:46:18--  https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
Resolving bin.equinox.io (bin.equinox.io)... 34.237.197.100, 52.86.175.38, 34.237.57.234, ...
Connecting to bin.equinox.io (bin.equinox.io)|34.237.197.100|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 13773305 (13M) [application/octet-stream]
Saving to: ‘ngrok-stable-linux-amd64.zip.1’

ngrok-stable-linux- 100%[===================>]  13.13M  18.9MB/s    in 0.7s    

2019-12-09 14:46:19 (18.9 MB/s) - ‘ngrok-stable-linux-amd64.zip.1’ saved [13773305/13773305]

Archive:  ngrok-stable-linux-amd64.zip
replace ngrok? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: ngrok                   
http://f8b50288.ngrok.io


#### Model building 및 fitting 시 TensorBoard 활용
- TensorBoard의 `log_dir`를 위에서 지정한 `LOG_DIR` 안에 실행 시간을 이름으로 하는 폴더로 지정. 
- `model.fit`을 실행한 후 위의 코드 실행 후 출력된 주소를 클릭하면 tensorboard가 실행됨.

In [7]:
## Your answer comes here.
from keras.callbacks import TensorBoard
from keras.models import Sequential
from keras import layers
import time 
now = time.strftime("%c")


model = Sequential()
model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(28, 28, 1)))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Flatten())
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(10, activation='softmax'))

model.compile(optimizer='rmsprop',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

callbacks_list = [
    TensorBoard(log_dir=LOG_DIR+'/'+now),
]

model.fit(train_images, train_labels, epochs=5, batch_size=128, validation_split=0.2, callbacks=callbacks_list)


Train on 48000 samples, validate on 12000 samples
Epoch 1/5
48000/48000 [==============================] - 35s 738us/step - loss: 0.2563 - acc: 0.9216 - val_loss: 0.0920 - val_acc: 0.9720

Epoch 2/5
48000/48000 [==============================] - 36s 745us/step - loss: 0.0709 - acc: 0.9782 - val_loss: 0.0552 - val_acc: 0.9849
Epoch 3/5
48000/48000 [==============================] - 35s 739us/step - loss: 0.0479 - acc: 0.9850 - val_loss: 0.0505 - val_acc: 0.9847
Epoch 4/5
48000/48000 [==============================] - 35s 732us/step - loss: 0.0358 - acc: 0.9888 - val_loss: 0.0457 - val_acc: 0.9860
Epoch 5/5
48000/48000 [==============================] - 35s 728us/step - loss: 0.0278 - acc: 0.9914 - val_loss: 0.0413 - val_acc: 0.9882
